### Step 4
# Evaluation of completion results



In [2]:
import sys, os
lib_dir = os.path.abspath(os.path.join(os.path.curdir, "lib/dispatcher/src"))
if lib_dir not in sys.path:
    print(f"Adding {lib_dir} to sys.path")
    sys.path.insert(0, lib_dir)
else:
    print(f"{lib_dir} already in sys.path")

os.environ["APP_CONFIG_DIR"] = os.path.abspath(os.path.join(os.path.curdir, "lib/dispatcher/config"))

data_path = os.path.abspath(os.path.join(os.path.curdir, "..", "..", "eval-data"))

import dotenv
dotenv.load_dotenv()

/Users/space/Documents/projects/REX/re-evaluate/experiments/re-test-template/lib/dispatcher/src already in sys.path


True

# Load the Responses

In [3]:
import json

qa_source_file = 'sample-occ-uce-mc-exam.json'

qa_input_file = os.path.join(data_path, "responses", qa_source_file)

with open(qa_input_file, 'r') as f:
    file_data = f.read()
    qa_data = json.loads(file_data)

print(f"Loaded {len(qa_data.get("eval-questions"))} responses from {qa_input_file}")



Loaded 3 responses from /Users/space/Documents/projects/REX/re-evaluate/eval-data/responses/sample-occ-uce-mc-exam.json


# Perform evals

In [4]:
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import AnswerRelevancyMetric


metric = AnswerRelevancyMetric(threshold=0.5)
test_cases = [LLMTestCase(
    input=response.get("question"),
    actual_output=response.get("response")
) for response in qa_data.get("eval-questions")]

# metric.measure(test_cases)
# print(metric.score)
# print(metric.reason)

score_data = evaluate(test_cases, [metric])
scores = [r.metrics_data[0].score for r in score_data.test_results]
print(scores)



✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 3 test case(s) in parallel: |██████████|100% (3/3) [Time Taken: 00:08,  2.94s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 0.8181818181818182, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.82 because the main response effectively identifies the benchmarks for measuring liquidity risk. However, introductory and unrelated interactive elements in the answer slightly diverge from the main focus, preventing it from reaching a perfect score. Keep up the great work; focusing on direct, targeted information can help achieve an even higher score!, error: None)

For test case:

  - input: Which two benchmarks are used to measure liquidity risk?
  - actual output: Hello, this is Erin, the enterprise assistant. To measure liquidity risk, the two main benchmarks used are:

1. Current Ratio: This is calculated by dividing a company's current assets by its current liabilities. It indicates a company's ability to pay its short-term obligations using its current assets.

2. Quick Ratio (Acid-Test Ratio): This is calculated by dividing a com

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.  
‼️ Looking for a place for your LLM test data to live 🏡? Use Confident AI for test reports, benchmark analysis, 
compare models/prompts, and catch regressions for your LLM system.

[0.8181818181818182, 0.9090909090909091, 0.9375]


In [7]:
for qa, s in zip(qa_data.get("eval-questions"), scores):
    qa["score-answer-relevance"] = s

output_file = os.path.join(data_path, "scores", f"{qa_source_file}.scored.json")

with open(output_file, 'w') as f:
    f.write(json.dumps(qa_data, indent=2))
    